In [1]:
import tensorflow as tf

In [171]:
from __future__ import absolute_import
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.request.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    raise Exception(
        'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)


# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = f.read(f.namelist()[0]).split()
  return data

words = read_data(filename)
print('Data size', len(words))

#Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])

data_index = 0


# Step 3: Function to generate a training batch for the skip-gram model.


#   tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
#   plot_only = 500
#   low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])
#   labels = [reverse_dictionary[i] for i in xrange(plot_only)]
#   plot_with_labels(low_dim_embs, labels)

# except ImportError:
#   print("Please install sklearn and matplotlib to visualize embeddings.")

Found and verified text8.zip
Data size 17005207
Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156]


In [3]:
tf.placeholder(tf.int32, shape=[batch_size])

<tf.Tensor 'Placeholder:0' shape=(128,) dtype=int32>

In [5]:
len(data)

17005207

In [9]:
len(data)

17005207

In [12]:
dictionary

{'fawn': 45848,
 'homomorphism': 9648,
 'nordisk': 39343,
 'nunnery': 36075,
 'chthonic': 33554,
 'sowell': 40562,
 'sonja': 38175,
 'showa': 32906,
 'woods': 6263,
 'hsv': 44222,
 'spiders': 14623,
 'hanging': 8021,
 'woody': 11150,
 'comically': 38935,
 'localized': 16716,
 'schlegel': 39763,
 'sevens': 47931,
 'canes': 30965,
 'sprague': 19496,
 'chatter': 45028,
 'orthographies': 37085,
 'nanking': 40997,
 'originality': 19567,
 'mutinies': 47932,
 'alphabetic': 13925,
 'hermann': 6482,
 'cytochrome': 26060,
 'brocklin': 43516,
 'stipulate': 42056,
 'eugenics': 7421,
 'cofinality': 36724,
 'danmarks': 46453,
 'capoeira': 8971,
 'appropriation': 21010,
 'holyrood': 49046,
 'taj': 29360,
 'strictest': 30374,
 'bringing': 3627,
 'wooded': 18967,
 'liaisons': 38573,
 'vibrational': 25367,
 'umts': 33555,
 'wooden': 4779,
 'wednesday': 13043,
 'circuitry': 14992,
 'elgar': 16456,
 'stereotypical': 15093,
 'immunities': 38936,
 'pantheistic': 30327,
 'thrace': 12647,
 'kublai': 30769,
 '

In [249]:
data_index = 0
reverse_dictionary = movieDictRev
data = iurw

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(random.choice(data[data_index]))#CHANGE!
    data_index = (data_index + 1) % len(data)
  #print(buffer)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      #print(buffer[target][skip_window],target)
      batch[i * num_skips + j] = buffer[target][skip_window]
      labels[i * num_skips + j, 0] = buffer[target][0]
    buffer.append(random.choice(data[data_index]))
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], '->', labels[i, 0])
  print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():
  vocabulary_size = len(data)
  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))################################***************

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  tf.initialize_all_variables().run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)##################################################*********
    feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
        print("")
  final_embeddings = normalized_embeddings.eval()

# # Step 6: Visualize the embeddings.

# def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
#   assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
#   plt.figure(figsize=(18, 18))  #in inches
#   for i, label in enumerate(labels):
#     x, y = low_dim_embs[i,:]
#     plt.scatter(x, y)
#     plt.annotate(label,
#                  xy=(x, y),
#                  xytext=(5, 2),
#                  textcoords='offset points',
#                  ha='right',
#                  va='bottom')

#   plt.savefig(filename)

# try:
#   from sklearn.manifold import TSNE
#   import matplotlib.pyplot as plt

650 -> 2
Glory (1989) -> Four Rooms (1995)
99 -> 0
Fargo (1996) -> Toy Story (1995)
117 -> 3
Twister (1996) -> Get Shorty (1995)
704 -> 1
Singin' in the Rain (1952) -> GoldenEye (1995)
650 -> 2
Glory (1989) -> Four Rooms (1995)
169 -> 4
Cinema Paradiso (1988) -> Copycat (1995)
261 -> 5
In the Company of Men (1997) -> Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
117 -> 3
Twister (1996) -> Get Shorty (1995)
Initialized
Average loss at step  0 :  170.755401611
Nearest to Much Ado About Nothing (1993): African Queen, The (1951), James and the Giant Peach (1996), Ballad of Narayama, The (Narayama Bushiko) (1958), Cosi (1996), 2 Days in the Valley (1996), Hard Target (1993), Short Cuts (1993), To Catch a Thief (1955),

Nearest to What's Eating Gilbert Grape (1993): Heathers (1989), Money Train (1995), Weekend at Bernie's (1989), My Fellow Americans (1996), Private Benjamin (1980), In the Line of Fire (1993), April Fool's Day (1986), Malice (1993),

Nearest to Legends of the Fall (199

In [174]:
data[0]

5239

In [22]:
dictionary["originated"]

3084

In [240]:
for i in dfMovie.index:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [1]:
import pandas as pd

In [ ]:
#http://grouplens.org/datasets/movielens/

#http://www.perozzi.net/publications/14_kdd_deepwalk.pdf

In [115]:
df=pd.read_table("./Desktop/ml-100k/u.data",header=None)

In [43]:
uMv = df.groupby(0).apply(lambda x: (x[2]))

In [113]:
df2=df.groupby(0).apply(lambda x:x).reset_index()

In [69]:
df2.set_index(0,inplace=True)

In [73]:
df2=df2[[1,2]]

In [77]:
df2.groupby(1).apply(lambda x: x[2]).as_matrix?

Object `as_matrix` not found.


In [99]:
import numpy as np
ui = np.zeros((len(set(df2.index)),len(set(df2[1]))))

In [122]:
import numpy as np
ui = np.zeros((len(set(df2.index)),len(set(df2[1]))))
def fillMat(i,j,k):
    ui[i-1,j-1]=float(df[2][k])
map(fillMat,df[0],df[1],xrange(len(df[0])))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [129]:
np.sum(ui,axis=1).shape

(943,)

In [126]:
iu=ui.T.dot(ui)

In [136]:
iu=ui.T.dot(ui)

def laplaceSmoothing(iu,addCoef = 1.0):
    return (iu+1)/(np.sum(iu,axis=1)+len(iu))

def randoWalks(iu,wpv,wl):
    rng = xrange(len(iu))
    wpvrng = xrange(wpv)
    wlrng = xrange(wl)
    def rw(idx):
        def repeatRw():
            rwLst = [idx]
            def w(nuIdx):
                rwLst.append(np.random.choice(rng,p=iu[nuIdx]/np.sum(iu[nuIdx])))
            map(w,wlrng)
            return rwLst
        return [repeatRw() for _ in wpvrng]
    return list(map(rw,rng))
    
        
iu=laplaceSmoothing(iu,addCoef = 1.0)        
iurw = randoWalks(iu,5,20)        

In [217]:
iurw = randoWalks(iu,5,20)

In [235]:
dfMovie.index

RangeIndex(start=0, stop=1682, step=1)

In [135]:
np.random.choice?

In [141]:
iurw[0][0]

[0, 747, 863, 32, 194]

In [144]:
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100 
np.random.choice(valid_window, valid_size, replace=False)

array([ 0, 52, 82,  1, 28, 50, 36, 89, 85, 46, 51, 70, 47, 19, 83, 37])

In [253]:
dfMovie=pd.read_csv("./Desktop/ml-100k/u.item",sep='|',header=None)

CParserError: Error tokenizing data. C error: Expected 1 fields in line 11, saw 2


In [149]:
dfMovie[0]=dfMovie[0]-1

In [248]:
from collections import defaultdict
movieDict = defaultdict(int)
movieDictRev= defaultdict(int)
cnt = 0
for i in list(dfMovie[1]):
    movieDict[i]=cnt
    movieDictRev[cnt]=i
    cnt+=1

In [245]:
movieDictRev = dict(zip(movieDict.values(),movieDict.keys()))

In [155]:
movieDictRev[0]

'Toy Story (1995)'

In [161]:
import random

In [168]:
random.choice(iurw[0])

[0, 747, 863, 32, 194]

In [212]:
random.choice(iurw[32])

[32, 719, 123, 487, 295]

In [215]:
valid_size

16

In [226]:
range(1000)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [263]:
mv1m=[]
with open("./Desktop/ml-1m/movies.dat") as f:
    for i in f.readlines():
        mv1m.append(i.split("::")[:2])

In [330]:
mv1mDict = {int(i[0]):i[1] for i in mv1m}
mv1mDictRev = dict(zip(mv1mDict.values(),mv1mDict.keys()))

In [271]:
df1m = pd.read_table("./Desktop/ml-1m/ratings.dat",sep="::",header=None)

/Users/itzme187/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [278]:
[i==j for i,j in zip(range(1,df1m[1].max()+1),sorted(list(set(df1m[1]))))]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 F

In [279]:
sorted(list(set(df1m[1])))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 110,
 111,
 112,
 113,
 114,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 1

In [281]:
list(df1m[1]).index(51)

ValueError: 51 is not in list

In [287]:
mv1mDict.pop(51)

'Guardian Angel (1994)'

In [288]:
mv1mDict[51]

KeyError: 51

In [318]:
from collections import defaultdict

def clean_dict(mv1mDict, df1m):
    newDict = defaultdict(int)
    cnt = 0
    chkLst = list(df1m[1])
    kyLst = sorted(list(mv1mDict.keys()))
    kyLst = list(map(lambda i: i-1,kyLst))
    for i in range(len(kyLst)):
        if kyLst[i]+cnt not in chkLst:
            kyLst=list(map(lambda i:i-1,kyLst))
            cnt+=1
        else:
            newDict[i-cnt]=mv1mDict[kyLst[i]+cnt]
    return newDict

In [319]:
nwdct=clean_dict(mv1mDict, df1m)

In [325]:
np.all([i==j for i,j in zip(sorted(nwdct.keys()),range(max(nwdct.keys())))])

True

In [328]:
print(len(set(df1m[1])))
print(len(nwdct.keys()))

3706
3650


In [337]:
import numpy as np
newMovDict = { i:(mv1mDict[j],j) for i,j in enumerate(sorted(list(set(df1m[1]))))}
revKey = {j[1]:i for i,j in zip(newMovDict.keys(),newMovDict.values())}
ui = np.zeros((len(set(df1m[0])),len(set(df1m[1]))))
def fillMat(i,j,k):
    ui[i-1,revKey[j]]=float(df1m[2][k])
map(fillMat,df1m[0],df1m[1],df1m.index)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [338]:
iu=ui.T.dot(ui)

def laplaceSmoothing(iu,addCoef = 1.0):
    return (iu+1)/(np.sum(iu,axis=1)+len(iu))

def randoWalks(iu,wpv,wl):
    rng = xrange(len(iu))
    wpvrng = xrange(wpv)
    wlrng = xrange(wl)
    def rw(idx):
        def repeatRw():
            rwLst = [idx]
            def w(nuIdx):
                rwLst.append(np.random.choice(rng,p=iu[nuIdx]/np.sum(iu[nuIdx])))
            map(w,wlrng)
            return rwLst
        return [repeatRw() for _ in wpvrng]
    return list(map(rw,rng))
    
        
iu=laplaceSmoothing(iu,addCoef = 1.0)        
iurw = randoWalks(iu,5,20)        

In [339]:
reverse_dictionary = {i:j[0] for i,j in newMovDict.items()}

In [340]:
reverse_dictionary[0]

'Toy Story (1995)'

In [341]:
data_index = 0
reverse_dictionary = {i:j[0] for i,j in newMovDict.items()}
data = iurw

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(random.choice(data[data_index]))#CHANGE!
    data_index = (data_index + 1) % len(data)
  #print(buffer)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      #print(buffer[target][skip_window],target)
      batch[i * num_skips + j] = buffer[target][skip_window]
      labels[i * num_skips + j, 0] = buffer[target][0]
    buffer.append(random.choice(data[data_index]))
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], '->', labels[i, 0])
  print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():
  vocabulary_size = len(data)
  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))################################***************

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  tf.initialize_all_variables().run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)##################################################*********
    feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
        print("")
  final_embeddings = normalized_embeddings.eval()

# # Step 6: Visualize the embeddings.

# def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
#   assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
#   plt.figure(figsize=(18, 18))  #in inches
#   for i, label in enumerate(labels):
#     x, y = low_dim_embs[i,:]
#     plt.scatter(x, y)
#     plt.annotate(label,
#                  xy=(x, y),
#                  xytext=(5, 2),
#                  textcoords='offset points',
#                  ha='right',
#                  va='bottom')

#   plt.savefig(filename)

# try:
#   from sklearn.manifold import TSNE
#   import matplotlib.pyplot as plt

3067 -> 0
Conceiving Ada (1997) -> Toy Story (1995)
2325 -> 2
Airport (1970) -> Grumpier Old Men (1995)
2017 -> 3
Modulations (1998) -> Waiting to Exhale (1995)
576 -> 1
Dances with Wolves (1990) -> Jumanji (1995)
2325 -> 2
Airport (1970) -> Grumpier Old Men (1995)
2191 -> 4
Babe: Pig in the City (1998) -> Father of the Bride Part II (1995)
2017 -> 3
Modulations (1998) -> Waiting to Exhale (1995)
3024 -> 5
Sister Act 2: Back in the Habit (1993) -> Heat (1995)
Initialized
Average loss at step  0 :  213.347717285
Nearest to Mortal Kombat (1995): Space Cowboys (2000), Gone with the Wind (1939), Shadow Conspiracy (1997), Big Carnival, The (1951), Cavalcade (1933), Mamma Roma (1962), Six Days Seven Nights (1998), 3 Strikes (2000),

Nearest to Wings of Courage (1995): Star Wars: Episode IV - A New Hope (1977), Entrapment (1999), Flipper (1996), Risky Business (1983), Surviving the Game (1994), Tomorrow Never Dies (1997), Century (1993), I, Worst of All (Yo, la peor de todas) (1990),

Nearest

In [351]:
i=237
valid_word =reverse_dictionary[i]
print(valid_word)

Gumby: The Movie (1995)


In [352]:
#valid_word = reverse_dictionary[valid_examples[i]]
top_k = 8 # number of nearest neighbors
nearest = (-sim[i, :]).argsort()[1:top_k+1]
log_str = "Nearest to %s:" % valid_word
for k in xrange(top_k):
    close_word = reverse_dictionary[nearest[k]]
    log_str = "%s %s," % (log_str, close_word)
print(log_str)
print("")

IndexError: index 237 is out of bounds for axis 0 with size 16

In [365]:
from scipy.spatial.distance import cosine

closest = np.argsort(map(lambda i: cosine(final_embeddings[13],final_embeddings[i]),range(len(final_embeddings))))

In [366]:
map(lambda i: print(reverse_dictionary[i]),closest[:10])

Nixon (1995)
Key Largo (1948)
Tron (1982)
Armageddon (1998)
Priest (1994)
F/X (1986)
Happiest Millionaire, The (1967)
Magnum Force (1973)
Singin' in the Rain (1952)
Trust (1990)


[None, None, None, None, None, None, None, None, None, None]

In [361]:
cosine?

In [362]:
reverse_dictionary.values()

['Toy Story (1995)',
 'Jumanji (1995)',
 'Grumpier Old Men (1995)',
 'Waiting to Exhale (1995)',
 'Father of the Bride Part II (1995)',
 'Heat (1995)',
 'Sabrina (1995)',
 'Tom and Huck (1995)',
 'Sudden Death (1995)',
 'GoldenEye (1995)',
 'American President, The (1995)',
 'Dracula: Dead and Loving It (1995)',
 'Balto (1995)',
 'Nixon (1995)',
 'Cutthroat Island (1995)',
 'Casino (1995)',
 'Sense and Sensibility (1995)',
 'Four Rooms (1995)',
 'Ace Ventura: When Nature Calls (1995)',
 'Money Train (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Assassins (1995)',
 'Powder (1995)',
 'Leaving Las Vegas (1995)',
 'Othello (1995)',
 'Now and Then (1995)',
 'Persuasion (1995)',
 'City of Lost Children, The (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Dangerous Minds (1995)',
 'Twelve Monkeys (1995)',
 'Wings of Courage (1995)',
 'Babe (1995)',
 'Carrington (1995)',
 'Dead Man Walking (1995)',
 'Across the Sea of Time (1995)',
 'It Takes Two (1995)',
 'Clueless (

In [367]:
import tensorflow as tf
import numpy as np

class Like2Vec(object):
    def __init(self,user_item,name_index_dict,axis = 0,add_coeff = 1.0):
        self.user_item = user_item
        self.axis_matrix = self.user_item.dot(self.user_item.T) if axis==0 else self.user_item.T.dot(self.user_item)
        self.final_matrix = self.laplace_smoothing(add_coeff)
        self.name_index_dict = name_index_dict
        self.index_name_dict = dict(zip(self.name_index_dict.values(),self.name_index_dict.keys()))
        self.data_index = 0
        
    def laplace_smoothing(self,add_coeff):
        return (self.axis_matrix+add_coeff)/(np.sum(self.axis_matrix,axis=1)+add_coeff*len(self.axis_matrix))
    
    def rando_walks(self,wpv,wl):
        rng = xrange(len(self.final_matrix))
        wpvrng = xrange(wpv)
        wlrng = xrange(wl)
        def rw(idx):
            def repeatRw():
                rwLst = [idx]
                def w(nuIdx):
                    rwLst.append(np.random.choice(rng,p=self.final_matrix[nuIdx]/np.sum(self.final_matrix[nuIdx])))
                map(w,wlrng)
                return rwLst
            return [repeatRw() for _ in wpvrng]
        return list(map(rw,rng))
    
    
    reverse_dictionary = {i:j[0] for i,j in newMovDict.items()}
    data = iurw

    def _generate_batch(self,batch_size, num_skips, skip_window):
      assert batch_size % num_skips == 0
      assert num_skips <= 2 * skip_window
      batch = np.ndarray(shape=(batch_size), dtype=np.int32)
      labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
      span = 2 * skip_window + 1 # [ skip_window target skip_window ]
      buffer = collections.deque(maxlen=span)
      for _ in range(span):
        buffer.append(random.choice(data[data_index]))#CHANGE!
        data_index = (data_index + 1) % len(data)

      for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
          while target in targets_to_avoid:
            target = random.randint(0, span - 1)
          targets_to_avoid.append(target)

          batch[i * num_skips + j] = buffer[target][skip_window]
          labels[i * num_skips + j, 0] = buffer[target][0]
        buffer.append(random.choice(data[data_index]))
        data_index = (data_index + 1) % len(data)
      return batch, labels

    def _build_skip_gram(self,batch_size = 128,embedding_size = 128,learn_rate=1.0,num_sampled = 64,num_skips = 2,
                         skip_window = 1,stddev=1.0,valid_size = 16 ,valid_window = 100 ):
        
         # Dimension of the embedding vector.
               # How many words to consider left and right.
                # How many times to reuse an input to generate a label.

        # We pick a random validation set to sample nearest neighbors. Here we limit the
        # validation samples to the words that have a low numeric ID, which by
        # construction are also the most frequent.
        # Random set of words to evaluate similarity on.
        # Only pick dev samples in the head of the distribution.
        valid_examples = np.random.choice(valid_window, valid_size, replace=False)
        # Number of negative examples to sample.

        graph = tf.Graph()

        with graph.as_default():
          vocabulary_size = len(data)
          # Input data.
          train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
          train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
          valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

          # Ops and variables pinned to the CPU because of missing GPU implementation
          with tf.device('/cpu:0'):
            # Look up embeddings for inputs.
            embeddings = tf.Variable(
                tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)

            # Construct the variables for the NCE loss
            nce_weights = tf.Variable(
                tf.truncated_normal([vocabulary_size, embedding_size],
                                    stddev=stddev / math.sqrt(embedding_size)))
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

          # Compute the average NCE loss for the batch.
          # tf.nce_loss automatically draws a new sample of the negative labels each
          # time we evaluate the loss.
          loss = tf.reduce_mean(
              tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                             num_sampled, vocabulary_size))################################***************

          # Construct the SGD optimizer using a learning rate of 1.0.
          optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)

          # Compute the cosine similarity between minibatch examples and all embeddings.
          norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
          normalized_embeddings = embeddings / norm
          valid_embeddings = tf.nn.embedding_lookup(
              normalized_embeddings, valid_dataset)
          similarity = tf.matmul(
              valid_embeddings, normalized_embeddings, transpose_b=True)
        
    def train(self,num_steps = 100001,verbose=False):
        def _print_loss(step):
            if step > 0:
                average_loss /= 2000
              # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            

        def _print_results():# Note that this is expensive (~20% slowdown if computed every 500 steps)
            
              sim = similarity.eval()
              for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                  close_word = reverse_dictionary[nearest[k]]
                  log_str = "%s %s," % (log_str, close_word)
                print(log_str)
                print("")

        with tf.Session(graph=graph) as session:
          # We must initialize all variables before we use them.
          tf.initialize_all_variables().run()
          #print("Initialized")

          average_loss = 0
          for step in xrange(num_steps):
            batch_inputs, batch_labels = generate_batch(
                batch_size, num_skips, skip_window)##################################################*********
            feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

            # We perform one update step by evaluating the optimizer op (including it
            # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
            
            
            if verbose:
                average_loss += loss_val
                
                if step % 2000 == 0:
                    _print_loss()
                    average_loss = 0
                if step % 10000 == 0:
                    _print_results()
                

            final_embeddings = normalized_embeddings.eval()

In [1]:
from sklearn.manifold import TSNE

In [1]:
import numpy as np

In [2]:
np.random.choice(100, 16, replace=False)

array([27, 65, 19, 53, 59, 72, 83, 74, 63, 48, 77, 24, 13, 78,  0, 55])